# 求解不含有不等式约束的线性方程组。


In [8]:
import scipy.io
import numpy as np
import scipy.linalg
# 读取 .mat 文件
mat = scipy.io.loadmat('EQ.mat')

# 查看 .mat 文件中的变量
print(mat.keys())

dict_keys(['__header__', '__version__', '__globals__', 'None', '__function_workspace__'])


In [9]:

for i in mat.keys():
    print(str(i)+str(mat[i]))

# # 提取系数矩阵 A 和常数向量 b
# A = mat[A_key]
# b = mat[b_key]

__header__b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Aug  2 11:10:48 2024'
__version__1.0
__globals__[]
None[(b'eqns', b'MCOS', b'sym', array([[3707764736],
        [         2],
        [         1],
        [         1],
        [         1],
        [         1]], dtype=uint32))           ]
__function_workspace__[[ 0  1 73 ...  0  0  0]]


In [10]:
# example
N = 3
A = np.array([range(N**2)]).reshape(N, N)
A = A - 1
b = np.array([range(N)])
b = b - 1
print(A)
print(b)

[[-1  0  1]
 [ 2  3  4]
 [ 5  6  7]]
[[-1  0  1]]


In [11]:
def solve_linear_system(A, b):
    b = b.flatten()
    # 判断 A 是否是方阵
    if A.shape[0] == A.shape[1]:
        try:
            # 尝试求解线性方程组 Ax = b
            x = scipy.linalg.solve(A, b)
            print("方程组有唯一解：", x)
        except np.linalg.LinAlgError as e:
            # 如果方程组无解或有无穷多解，抛出异常
            print("方程组无解或有无穷多解。错误信息：", e)
    else:
        # 使用最小二乘法求解方程组 Ax = b
        x, residuals, rank, s = scipy.linalg.lstsq(A, b)
        print("方程组的最小二乘解：", x)
        if residuals.size > 0 and np.all(residuals < 1e-10):
            print("最小二乘解是唯一的。")
        else:
            print("最小二乘解不是唯一的。")

In [12]:
# 使用示例
solve_linear_system(A, b)

方程组有唯一解： [ 0.375       0.58333333 -0.625     ]


/tmp/ipykernel_2461/3977183546.py:7: LinAlgWarning: Ill-conditioned matrix (rcond=9.25186e-18): result may not be accurate.
  x = scipy.linalg.solve(A, b)


# 求解含有不等式约束的线性方程组。

In [13]:
import numpy as np
from scipy.optimize import linprog

def solve_linear_system(A, b, C, d):
    """
    求解含有不等式约束的线性方程组。
    
    参数:
    A (np.ndarray): 方程系数矩阵
    b (np.ndarray): 方程常数向量
    C (np.ndarray): 不等式系数矩阵
    d (np.ndarray): 不等式常数向量
    
    返回:
    tuple: (是否有解, 解的值) 其中“是否有解”为布尔值，解的值为数组（如果有解的话）
    """
    # 目标函数系数，这里我们只是为了求解，所以设置为零
    c = np.zeros(A.shape[1])
    
    # 求解线性规划问题
    result = linprog(c, A_ub=C, b_ub=d, A_eq=A, b_eq=b, method='highs')
    
    if result.success:
        return (True, result.x)
    else:
        return (False, None)


In [14]:

# 示例：维数为3的线性方程组和不等式约束

# 方程系数矩阵 A (3x3)
A = np.array([
    [1, 1, 1],
    [2, 1, 1],
    [1, 2, 3]
])

# 方程常数向量 b
b = np.array([6, 8, 13])

# 不等式系数矩阵 C (2x3)
C = np.array([
    [-1, 0, 0],
    [0, -1, -1]
])

# 不等式常数向量 d
d = np.array([-2, -3])

# 求解线性方程组
has_solution, solution = solve_linear_system(A, b, C, d)

if has_solution:
    print("存在解，解为:", solution)
else:
    print("没有解")


存在解，解为: [2. 1. 3.]



### 解释：

1. **函数定义**：
   - `solve_linear_system` 函数可以接受任意维数的矩阵和向量，只要它们的维度匹配。

2. **示例**：
   - **方程系数矩阵 `A`** 是一个 \(3 \times 3\) 矩阵，表示有3个方程和3个变量。
   - **方程常数向量 `b`** 是一个长度为3的向量。
   - **不等式系数矩阵 `C`** 是一个 \(2 \times 3\) 矩阵，表示有2个不等式约束和3个变量。
   - **不等式常数向量 `d`** 是一个长度为2的向量。

   - 在这个例子中，方程组为：
     \[
     \begin{cases}
     x_1 + x_2 + x_3 = 6 \\
     2x_1 + x_2 + x_3 = 8 \\
     x_1 + 2x_2 + 3x_3 = 13
     \end{cases}
     \]
     不等式约束为：
     \[
     \begin{cases}
     x_1 \leq 2 \\
     x_2 + x_3 \leq 3
     \end{cases}
     \]

   - 调用 `solve_linear_system` 函数来求解这个方程组和约束，并输出结果。

通过这种方式，你可以处理任意维数的线性方程组和不等式约束。